# LAB 4 : Collaborative Filtering Memory-based

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/xlab-recommendation/blob/main/solution_notebooks/04_collaborative_filtering_memory_based.ipynb)

* Dataset: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations
* Objective: To find nearest neighbors of user and item 
* Libraries : Pandas, Scipy, Numpy, Sklearn
* Use 'transactions.csv' to find user similarity and use 'articles.csv' to find item similarity

## User Similarity

### Read dataset for find user similarity

In [ ]:
! pip install pandas
! pip install scipy
! pip install numpy
! pip install scikit-learn
! pip install gdown

In [ ]:
# import essential library
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os.path as op
import os
import gdown

In [ ]:
# download folder from google drive
url = "https://drive.google.com/drive/folders/1N8ZZVk0carCvy3mTGwbZ4IZpZ6cOFo5j?usp=drive_link"
gdown.download_folder(url, use_cookies=False)

In [ ]:
# read dataset for find user similarity
file_name = "data/transactions.csv"
transactions_path = op.join(os.getcwd(), file_name)
transactions_df = pd.read_csv(transactions_path)

In [ ]:
# check transactions_df
transactions_df.head()

### Select data for find user similarity

In [ ]:
# select some data from transactions_df
selected_user_df = transactions_df[["customer_id", "article_id"]].sort_values(
    "customer_id", ignore_index=True
)

In [ ]:
# check selected_user_df
selected_user_df.head()

In [ ]:
# find unique customer_id from selected_user_df
print(f"unique customer_id : {len(pd.unique(selected_user_df['customer_id']))}")
# find unique article_id from selected_user_df
print(f"unique article_id : {len(pd.unique(selected_user_df['article_id']))}")

### Select data that customer have transaction greater than 500

In [ ]:
# select data that customer have transaction greater than 500
customer_counts = selected_user_df["customer_id"].value_counts()
filtered_df = selected_user_df.loc[
    selected_user_df["customer_id"].isin(customer_counts[customer_counts > 500].index)
].drop_duplicates(["customer_id", "article_id"], ignore_index=True)

In [ ]:
# find unique customer_id from filtered_df
print(f"unique customer_id : {len(pd.unique(filtered_df['customer_id']))}")
# find unique article_id from filtered_df
print(f"unique article_id : {len(pd.unique(filtered_df['article_id']))}")

In [ ]:
# astype customer_id, article_id as catagory
filtered_df["customer_id"] = filtered_df["customer_id"].astype("category")
filtered_df["article_id"] = filtered_df["article_id"].astype("category")

# add interact to filtered_df
filtered_df["interact"] = 1

In [ ]:
# check filtered_df
filtered_df.head()

### Create user-item matrix by using sparse matrix

In [ ]:
X_user = csr_matrix(
    (
        filtered_df["interact"],
        (filtered_df["customer_id"].cat.codes, filtered_df["article_id"].cat.codes),
    )
)

### Find nearest neighbors of the user

In [ ]:
k_user = 2  # Number of nearest neighbors to find
neighbors_model_user = NearestNeighbors(
    n_neighbors=k_user, metric="cosine"
)  # You can change the metric as needed

# Fit the model on your DataFrame
neighbors_model_user.fit(X_user)

In [ ]:
# Find the nearest neighbors for a specific data point (e.g., row 0)
row_index_user = (
    0  # You can change cat_code of customer that you want to find nearest neighbors
)
query_point_user = X_user.toarray()[row_index_user].reshape(
    1, -1
)  # Convert the row to a 2D array
distances_user, indices_user = neighbors_model_user.kneighbors(query_point_user)

print(distances_user, indices_user)

### Recommend item from nearest neighbor to customer

In [ ]:
# change cat code to customer_id for customer
customer_id = pd.Categorical.from_codes([row_index_user], categories=filtered_df["customer_id"].cat.categories).astype(str)[0]
customer_id

In [ ]:
# change cat code to customer_id for near_customer
near_customer_id = pd.Categorical.from_codes(indices_user[0], categories=filtered_df["customer_id"].cat.categories).astype(str)
near_customer_id

In [ ]:
# find nearest_customer_id
nearest_customer_id = list(set(near_customer_id) - set([customer_id]))[0]
nearest_customer_id

In [ ]:
# find article_id that customer_id buy
customer_buy = filtered_df[filtered_df['customer_id'] ==  customer_id]['article_id'].values
customer_buy

In [ ]:
# find article_id that nearest_customer_id buy
nearest_customer_buy = filtered_df[filtered_df['customer_id'] ==  nearest_customer_id]['article_id'].values
nearest_customer_buy

In [ ]:
# recommend article that nearest customer buy but customer doesn't buy to customer
recommend_item = list(set(nearest_customer_buy) - set(customer_buy))
recommend_item

## Item Similarity

### Read dataset for find item similarity

In [ ]:
# read dataset for find item similarity
file_name = "data/articles.csv"
article_path = op.join(os.getcwd(), file_name)
article_df = pd.read_csv(article_path)

In [ ]:
# check article_df
article_df.head()

### Select data for find item similarity

In [ ]:
# select some data from article dataframe
selected_item_column = [
    "product_type_no",
    "graphical_appearance_no",
    "colour_group_code",
    "index_group_no",
    "section_no",
    "garment_group_no",
]
selected_item_df = article_df[selected_item_column]

In [ ]:
# check selected_item_df
selected_item_df.head()

### Use one hot technique

In [ ]:
dummies_df = pd.get_dummies(selected_item_df, columns=selected_item_column)
dummies_df.head()

### Find nearest neighbor of the item

In [ ]:
k_article = 5  # Number of nearest neighbors to find
neighbor_model_article = NearestNeighbors(
    n_neighbors=k_article, metric="cosine"
)  # You can change the metric as needed

# Fit the model on your DataFrame
neighbor_model_article.fit(dummies_df)

In [ ]:
# Find the nearest neighbors for a specific data point (e.g., row 0)
row_index_article = (
    100  # You can change index of article that you want to find nearest neighbors
)
query_point_article = dummies_df.iloc[row_index_article].values.reshape(
    1, -1
)  # Convert the row to a 2D array
distances_article, indices_article = neighbor_model_article.kneighbors(
    query_point_article
)

# Print the nearest neighbors and their distances
nearest_article_id = article_df.iloc[indices_article[0]]["article_id"].values
print(f"Distances: {distances_article}")
print(f"Nearest Neighbors: {nearest_article_id}")